In [7]:
import yfinance as yf
import pandas as pd
import requests
from datetime import datetime
import mplfinance as mpf

In [1]:
def fetch_stock_data(ticker):
    """Fetches historical stock data for the given ticker from Yahoo Finance API"""
    try:
        today = datetime.date.today().strftime("%Y-%m-%d")
        data = yf.download(ticker, start="1900-01-01", end=today, group_by="ticker")
        if data.empty:
            raise Exception("No data available for the given ticker.")
        else:
            data.columns = data.columns.droplevel(0)
            data.columns = ["Open", "High", "Low", "Close", "Volume"]
            return data
    except Exception as e:
        print(f"Error fetching stock data for {ticker}: {str(e)}")
        return None

In [33]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2010-01-04,6.422878,6.455078,6.391279,6.440332,493729600
2010-01-05,6.458088,6.487881,6.417461,6.451468,601904800
2010-01-06,6.451465,6.477045,6.342225,6.348846,552160000
2010-01-07,6.372319,6.379843,6.291067,6.337110,477131200
2010-01-08,6.328683,6.379842,6.291368,6.379240,447610800
...,...,...,...,...,...
2025-03-05,235.419998,236.550003,229.229996,235.740005,47227600
2025-03-06,234.440002,237.860001,233.160004,235.330002,45170400
2025-03-07,235.110001,241.369995,234.759995,239.070007,46273600
